Historical codebook: http://web.mit.edu/cstewart/www/data/cb9999

In [146]:
library(tidyverse)

In [168]:
prep_committees <- function(type){
    path <- paste0(type, "_committees_modern.csv")
    df <- read_csv(path)
    
    if(type == "house"){
        df <- df %>% 
            rename(`Party Code` = Party,
                   `Committee Code` = `Committee code`)
    }
    
    df <- df %>% 
        select(party = `Party Code`, 
               id = `ID #`, 
               committee = `Committee Code`, 
               session = `Congress`,
               name = `Name`)
    
    df <- df %>% 
        separate(col = name, sep = ", ", into = c("last", "first")) %>% 
        separate(col = first, sep = " ", into = c("first", "other")) %>% 
        filter(!is.na(id)) %>% 
        mutate(party = case_when(party == 100 ~ 1,
                                 party == 200 ~ 2, 
                                 TRUE ~ 1)) %>% 
        filter(party > 0)
    
    lookup <- df %>% 
        group_by(id, first, last, party) %>% 
        filter(row_number() == 1) %>% 
        ungroup() %>% 
        mutate(new_id = row_number()) %>% 
        select(id, new_id, first, last, party)

    df <- df %>% 
        left_join(lookup)
    
    out_path <- paste0(type, "_committees.csv")
    
    df %>% write_csv(out_path)
} 

In [169]:
prep_committees("house")

Warning message:
“Missing column names filled in: 'X21' [21]”
Parsed with column specification:
cols(
  .default = col_double(),
  Name = col_character(),
  `Date of Assignment` = col_character(),
  `Date of Termination` = col_character(),
  `Appointment Citation` = col_character(),
  `Committee Name` = col_character(),
  `State Name` = col_character(),
  Notes = col_character(),
  X21 = col_logical()
)

See spec(...) for full column specifications.

Warning message:
“Expected 2 pieces. Additional pieces discarded in 488 rows [2, 27, 61, 83, 86, 113, 123, 155, 178, 228, 230, 243, 274, 323, 343, 351, 373, 412, 433, 450, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 2 rows [9062, 9140].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 292 rows [32, 81, 304, 319, 330, 375, 514, 571, 591, 687, 811, 886, 939, 1078, 1081, 1127, 1222, 1286, 1305, 1340, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 6781 rows 

In [151]:
senate_df <- read_csv("senate_committees_modern.csv")
senate_df <- senate_df %>% 
    select(party = `Party Code`, 
           id = `ID #`, 
           committee = `Committee Code`, 
           session = `Congress`,
           name = `Name`)

Warning message:
“Missing column names filled in: 'X10' [10], 'X22' [22], 'X23' [23], 'X24' [24], 'X25' [25]”
Parsed with column specification:
cols(
  .default = col_double(),
  Name = col_character(),
  `Date of Appointment` = col_character(),
  `Date of Termination` = col_character(),
  X10 = col_logical(),
  `Appointment Citation` = col_character(),
  `Committee Name` = col_character(),
  `State Name` = col_character(),
  Notes = col_character(),
  X22 = col_logical(),
  X23 = col_logical(),
  X24 = col_logical(),
  X25 = col_logical()
)

See spec(...) for full column specifications.



In [152]:
senate_df <- senate_df %>% 
    separate(col = name, sep = ", ", into = c("last", "first")) %>% 
    separate(col = first, sep = " ", into = c("first", "other")) %>% 
    filter(!is.na(id)) %>% 
    mutate(party = case_when(party == 100 ~ 1,
                             party == 200 ~ 2, 
                             TRUE ~ 1)) %>% 
    filter(party > 0)

Warning message:
“Expected 2 pieces. Additional pieces discarded in 155 rows [69, 71, 86, 91, 96, 113, 114, 169, 176, 177, 182, 191, 193, 218, 224, 245, 265, 297, 386, 475, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 1 rows [3660].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 162 rows [1258, 1266, 1319, 1424, 1458, 1498, 1579, 1629, 1630, 1644, 1650, 1663, 1675, 1712, 1722, 1785, 1821, 1860, 1899, 1968, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 3100 rows [2, 4, 5, 6, 8, 12, 13, 14, 15, 17, 24, 25, 26, 28, 30, 33, 34, 36, 37, 39, ...].”


In [153]:
lookup <- senate_df %>% 
    group_by(id, first, last, party) %>% 
    filter(row_number() == 1) %>% 
    ungroup() %>% 
    mutate(new_id = row_number()) %>% 
    select(id, new_id, first, last, party)

senate_df <- senate_df %>% 
    left_join(lookup)

Joining, by = c("party", "id", "last", "first")



In [155]:
senate_df %>% 
    group_by(session) %>% 
    summarise(n = n())

`summarise()` ungrouping output (override with `.groups` argument)



session,n
<dbl>,<int>
103,393
104,405
105,383
106,389
107,429
108,417
109,404
110,431
111,443


In [156]:
df %>% 
    write_csv("senate_committees.csv")